In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/local/Cellar/openjdk/17/libexec/openjdk.jdk/Contents/Home"


In [21]:
# Load 43 test topics for which qrels exist
import csv

with open('collections/queries.200.tsv','r') as tsvIn:
    reader = csv.reader(tsvIn, delimiter='\t')
    
    topics43 = {int(row[0]): {"title": row[1]} for row in reader}
    
    print(len(topics43))

43


In [10]:
# Load msmarco-passage index into Pyserini searcher object
from pyserini.search import SimpleSearcher

searcher = SimpleSearcher.from_prebuilt_index('msmarco-passage')

Attempting to initialize pre-built index msmarco-passage.


index-msmarco-passage-20201117-f87c94.tar.gz: 2.07GB [53:24, 692kB/s]                                                             


Extracting /Users/louiseleibbrandt/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.tar.gz into /Users/louiseleibbrandt/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.1efad4f1ae6a77e235042eff4be1612d...
Initializing msmarco-passage...


In [29]:
# Run search at rank 1000
def run_all_queries(file, topics, searcher):
    with open(file, 'w') as runfile:
        cnt = 0
        print('Running {} queries in total'.format(len(topics)))
        for id in topics:
            query = topics[id]['title']
            hits = searcher.search(query, 1000)
            for i in range(0, len(hits)):
                _ = runfile.write('{} Q0 {} {} {:.6f} Anserini\n'.format(id, hits[i].docid, i+1, hits[i].score))
            cnt += 1
            if cnt % 100 == 0:
                print(f'{cnt} queries completed')


# Set BM25 parameters to default values
searcher.set_bm25(0.9, 0.4)
run_all_queries('runs/run-msmarco-43testqueries-bm25.txt', topics43, searcher)

Running 43 queries in total


In [30]:
# NDCG (from trec_eval) for different ranks, rank 10 (second line) is used in our report
!python -m pyserini.eval.trec_eval -c -mndcg_cut collections/2019qrels-pass.txt runs/run-msmarco-43testqueries-bm25.txt

/Users/louiseleibbrandt/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/Users/louiseleibbrandt/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-mndcg_cut', 'collections/2019qrels-pass.txt', 'runs/run-msmarco-43testqueries-bm25.txt']
Results:
ndcg_cut_5            	all	0.5278
ndcg_cut_10           	all	0.5058
ndcg_cut_15           	all	0.4980
ndcg_cut_20           	all	0.4914
ndcg_cut_30           	all	0.4884
ndcg_cut_100          	all	0.5018
ndcg_cut_200          	all	0.5332
ndcg_cut_500          	all	0.5813
ndcg_cut_1000         	all	0.6067

